<a href="https://colab.research.google.com/github/randywreed/DigiJesus-GPT3/blob/main/digitaljesus_prompt_gpt_neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.5MB/s 
     |████████████████████████████████| 3.3MB 46.8MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 


In [ ]:
import os
import transformers


In [ ]:
#if not using gdrive
modelpath="EleutherAI/gpt-neo-1.3B"
tokenpath="EleutherAI/gpt-neo-1.3B"

In [ ]:
#skip if not using Gdrive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#if using Gdrive (probably requires colab pro get's out of memory error)
modelpath='/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomodel/model'
tokenpath='/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomodel/tokenizer'


In [ ]:
# download pretrained model and save to google drive. 
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model.save_pretrained(modelpath)
tokenizer.save_pretrained(tokenpath)

('/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/tokenizer_config.json',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/special_tokens_map.json',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/vocab.json',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/merges.txt',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/added_tokens.json')

In [ ]:
from transformers import pipeline
generator=pipeline('text-generation', model="EleutherAI/gpt-neo-1.3B",device=0)

In [ ]:
from transformers import GPT2Tokenizer
tokenizer=GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B",device=0)

In [ ]:
from transformers import pipeline, GPT2Tokenizer
generator = pipeline('text-generation', model="EleutherAI/gpt-neo-2.7B", device=0)


In [ ]:
from transformers import GPT2Tokenizer
tokenizer=GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B", device=0)

In [ ]:
import sys
from io import StringIO
from IPython import get_ipython


class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        # print("exiting")  # optionally print some message to stdout, too
        # ... or do other stuff before exit
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup


def ipy_exit():
    raise IpyExit


if get_ipython():    # ...run with IPython
    exit = ipy_exit  # rebind to custom exit
else:
    exit = exit      # just make exit importable

In [ ]:
import torch

with open("prompt.txt","r+") as promptfile:
    initprompt=promptfile.read()
    newpassflag=True
    while newpassflag:
        newpassage=input('enter new passage: ')
        prompt=initprompt+"\npassage:"+newpassage+"\nquote:"
        print(prompt)
        loopflag=True
        while loopflag:        
            minl=tokenizer(prompt, return_length=True)
            print(f'prompt length={minl.length}')
            passlen=tokenizer(newpassage,return_length=True)
            print(f'new prompt token length {passlen.length}')
            minl=int(input('enter max length: '))
            temparature=float(input('Enter tempreature: '))
            response=generator(prompt, do_sample=True,bs=2 ,max_length=minl, temperature=temparature)
            newresponse=response[0]['generated_text'].split("###")[-2]
            newsayings=newresponse.split('\n')
            print(f'prompt= {newsayings[1]}')
            print(f'quote={newsayings[2]}')
           
            #gc.collect()
            torch.cuda.empty_cache()
            #print(response)
            action=input('type correction, hit return to retry, a to add, x to stop, r to re-enter: ')
            if len(action)>1:
                promptfile.write(action+"\n###\n")
            else:
                if action=="a":
                    #print(response[0])
                    newprompt="\n"+newsayings[1]+"\n"+ newsayings[2]+"\n###"
                    import pdb; pdb.set_trace()
                    promptfile.write(newprompt)
                    loopflag=False
                    getmore=input('enter c to continue or x to exit')
                    if getmore=="x":
                        promptfile.close()
                        exit()
                else:
                    if action=="x":
                        promptfile.close()
                        exit()
                    else:
                        if action=="r":
                            loopflag=False

enter new passage:  and Jesus said, I did not come to bring peace but a sword


extract quote from passage
passage: But Jesus answered, Permit it now ; for thus must we do to accomplish all that is right. Then he permitted him.
quote: Permit it now; for thus must we do to accomplish all that is right.
###
passage: But Jesus answered, It is written, Man shall not live by bread alone, but in whatever way God may ordain.
quote: It is written, Man shall not live by bread alone, but in whatever way God may ordain.
###
passage: Jesus said to him, It is written also, Thou shalt not make trial of the LoRD, thy God.
quote: It is written also, Thou shalt not make trial of the LoRD, thy God.
###
passage: FROM that time Jesus began to preach, saying, Reform; for the kingdom of Heaven is at hand.
quote: Reform; for the kingdom of Heaven is at hand.
###
passage: And lo. they brought to him a man with a palsy, lying on a bed. And Jesus, perceiving their faith, said to the paralytic, Take courage, s son your sins have been forgiven.
quote: Take courage, s son your sins have been 

enter max length:  1325
Enter tempreature:  .9


prompt= passage:and Jesus said, I did not come to bring peace but a sword
quote=quote: I did not come to bring peace but a sword.


type correction, hit return to retry, a to add, x to stop, r to re-enter:  a


...

> <ipython-input-9-2e8af0ec82c0>(37)<module>()
     35                     promptfile.write(newprompt)
     36                     loopflag=False
---> 37                     getmore=input('enter c to continue or x to exit')
     38                     if getmore=="x":
     39                         promptfile.close()



ipdb>  r
enter c to continue or x to exit c
enter new passage:  And turning to him, Jesus answered, I do not speak on my own, but it is my father who speaks.


extract quote from passage
passage: But Jesus answered, Permit it now ; for thus must we do to accomplish all that is right. Then he permitted him.
quote: Permit it now; for thus must we do to accomplish all that is right.
###
passage: But Jesus answered, It is written, Man shall not live by bread alone, but in whatever way God may ordain.
quote: It is written, Man shall not live by bread alone, but in whatever way God may ordain.
###
passage: Jesus said to him, It is written also, Thou shalt not make trial of the LoRD, thy God.
quote: It is written also, Thou shalt not make trial of the LoRD, thy God.
###
passage: FROM that time Jesus began to preach, saying, Reform; for the kingdom of Heaven is at hand.
quote: Reform; for the kingdom of Heaven is at hand.
###
passage: And lo. they brought to him a man with a palsy, lying on a bed. And Jesus, perceiving their faith, said to the paralytic, Take courage, s son your sins have been forgiven.
quote: Take courage, s son your sins have been 

enter max length:  1342
Enter tempreature:  .9


prompt= passage:And turning to him, Jesus answered, I do not speak on my own, but it is my father who speaks.
quote=quote: I do not speak on my own, but it is my father who speaks.


In [ ]:
del generator
#gc.collect()
torch.cuda.empty_cache()